## Instalación de Librerías

Usando Anaconda Prompt se debe usar los siguientes comandos para importar la librería de Keras.
<br>Link: https://medium.com/@pushkarmandot/installing-tensorflow-theano-and-keras-in-spyder-84de7eb0f0df
<br>TensorFlow: https://www.tensorflow.org/install/
<br>Link: https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/

```conda create -n tensorflow-gpu pip python=3.5```
<br>```conda activate tensorflow-gpu```
<br>```conda install keras ```

Otra opción es la siguiente:
<br>Entrar en modo administrador a Anaconda Prompt e introducir los siguientes comandos.
<br>```conda update conda ```
<br>```conda install keras ```

Añadir en el path las librerias importadas si es que estás en Windows:

``` set path=%PATH%;C:\Users\Alvaro\AppData\Roaming\Python\Python35\Scripts ```

Si está usando Linux, usar los siguientes comandos.

In [ ]:
#!pip install cython --user
#!pip install --force-reinstall regex==2017.04.5
#!pip install pathlib --user
#!pip install msgpack --user
!pip install tensorflow-gpu --user
!pip install keras --user

Pruebo la correcta importación de librerías.

In [1]:
import tensorflow as tf
hello = tf.constant("Hello, TF!")
sess = tf.Session()
print(sess.run(hello))

C:\Users\alirapal\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


b'Hello, TF!'


In [2]:
a = tf.constant(10)
b = tf.constant(32)
print(sess.run(a + b))

42


In [3]:
import keras

Using TensorFlow backend.


## Procesamiento

In [4]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
import pickle

In [5]:
#Archivo de texto 
DATA_DIR = "./warpeace_input.txt" 
#Modificar BATCH_SIZE o HIDDEN_DIM en caso tengan problemas de memoria
BATCH_SIZE = 50 
HIDDEN_DIM = 250 #500
#Parametro para longitud de secuencia a analizar
SEQ_LENGTH = 50 
#Parametro para cargar un pesos previamente entrenados (checkpoint)
WEIGHTS = '' 

#Parametro para indicar cuantos caracteres generar en cada prueba
GENERATE_LENGTH = 500 
#Parametros para la red neuronal
LAYER_NUM = 2 
NB_EPOCH = 20

**Función A:
<br>(1) Carga de un archivo de texto, (2) Construcción de estructuras de entrada y salida de la red**

In [6]:
# method for preparing the training data
def load_data(data_dir, seq_length):
    #Carga del archivo
    data = open(data_dir, 'r').read()
    #Caracteres unicos
    chars = list(set(data))
    VOCAB_SIZE = len(chars)

    print('Data length: {} characters'.format(len(data)))
    print('Vocabulary size: {} characters'.format(VOCAB_SIZE))
    print(chars)
    
    #Indexacion de los caracteres
    ix_to_char = {ix:char for ix, char in enumerate(chars)}
    char_to_ix = {char:ix for ix, char in enumerate(chars)}
    
    #Estructuras de entrada y salida
    NUMBER_OF_SEQ = int(len(data)/seq_length)
    print('Number of sequences: {}'.format(NUMBER_OF_SEQ))
    X = np.zeros((NUMBER_OF_SEQ, seq_length, VOCAB_SIZE))
    y = np.zeros((NUMBER_OF_SEQ, seq_length, VOCAB_SIZE))
    
    for i in range(0, NUMBER_OF_SEQ):
        #LLenado de la estructura de entrada X
        X_sequence = data[i*seq_length:(i+1)*seq_length]
        X_sequence_ix = [char_to_ix[value] for value in X_sequence]
        #one-hot-vector (input)
        input_sequence = np.zeros((seq_length, VOCAB_SIZE))  
        #uso del diccionario para completar el one-hot-vector
        for j in range(seq_length):
            input_sequence[j][X_sequence_ix[j]] = 1.
            X[i] = input_sequence
            
        #Llenado de la estructura de salida y
        y_sequence = data[i*seq_length+1:(i+1)*seq_length+1]
        y_sequence_ix = [char_to_ix[value] for value in y_sequence]
        #one-hot-vector (output)
        target_sequence = np.zeros((seq_length, VOCAB_SIZE))
        #uso del diccionario para completar el one-hot-vector
        for j in range(seq_length):
            target_sequence[j][y_sequence_ix[j]] = 1.
            y[i] = target_sequence
            
    return X, y, VOCAB_SIZE, ix_to_char

**Función B:
<br>Generación de textos**

In [9]:
# method for generating text
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

## Entrenamiento y Prueba

**Uso de la Función A: carga de los datos**

In [10]:
# Creating training data
X, y, VOCAB_SIZE, ix_to_char = load_data(DATA_DIR, SEQ_LENGTH)

Data length: 3196232 characters
Vocabulary size: 86 characters
['.', '2', 'v', 'a', 'B', '!', '(', 'W', 'u', 'q', 'o', 'U', 'b', 'D', 'z', 'f', 'n', '¤', ' ', 'r', '?', 'C', 't', '9', '/', 'k', 'Q', 'S', '\xa0', 'x', 'Z', 'L', '»', '©', 'N', 'K', '*', 'ï', "'", 'y', 'I', '¿', 'l', ';', ':', 's', 'V', 'A', 'g', 'G', '0', 'i', 'Ã', '4', 'Y', 'M', '=', 'H', '1', 'w', 'j', '6', '"', 'P', '3', '-', '5', 'd', 'm', 'J', ',', '8', 'h', 'T', 'F', '\n', 'c', 'p', ')', 'X', 'ª', 'O', 'R', 'e', '7', 'E']
Number of sequences: 63924


**Es importante guardar el diccionario `ix_to_char` en un archivo binario. Este debe ser cargado cada vez que se quiera retomar el entrenamiento o generar texto a partir de un checkpoint, debido a que el orden de los caracteres en el diccionario podría modificarse (no es un orden fijo)**
<br>**NO MODIFICAR ESTE PICKLE AL REINICIAR EL NOTEBOOK PARA PROBAR CHECKPOINTS**

# ALERTA: NO EJECUTAR LAS 3 SIGUIENTES LÍNEA SI ES QUE YA EXISTEN CHECKPOINTS Y EL IX_TO_CHAR

In [11]:
#No modificar el pickle al reiniciar el cuaderno de trabajo para probar checkpoints previos
with open('ix_to_char.pickle', 'wb') as handle:
    pickle.dump(ix_to_char, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
print(ix_to_char)

{0: '.', 1: '2', 2: 'v', 3: 'a', 4: 'B', 5: '!', 6: '(', 7: 'W', 8: 'u', 9: 'q', 10: 'o', 11: 'U', 12: 'b', 13: 'D', 14: 'z', 15: 'f', 16: 'n', 17: '¤', 18: ' ', 19: 'r', 20: '?', 21: 'C', 22: 't', 23: '9', 24: '/', 25: 'k', 26: 'Q', 27: 'S', 28: '\xa0', 29: 'x', 30: 'Z', 31: 'L', 32: '»', 33: '©', 34: 'N', 35: 'K', 36: '*', 37: 'ï', 38: "'", 39: 'y', 40: 'I', 41: '¿', 42: 'l', 43: ';', 44: ':', 45: 's', 46: 'V', 47: 'A', 48: 'g', 49: 'G', 50: '0', 51: 'i', 52: 'Ã', 53: '4', 54: 'Y', 55: 'M', 56: '=', 57: 'H', 58: '1', 59: 'w', 60: 'j', 61: '6', 62: '"', 63: 'P', 64: '3', 65: '-', 66: '5', 67: 'd', 68: 'm', 69: 'J', 70: ',', 71: '8', 72: 'h', 73: 'T', 74: 'F', 75: '\n', 76: 'c', 77: 'p', 78: ')', 79: 'X', 80: 'ª', 81: 'O', 82: 'R', 83: 'e', 84: '7', 85: 'E'}


In [13]:
print(X.shape, y.shape, VOCAB_SIZE)

(63924, 50, 86) (63924, 50, 86) 86


### Creación de la RNN (LSTM)

In [14]:
# Creating and compiling the Network
model = Sequential()

#Añadiendo las capas LSTM
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
#Añadiendo la operacion de salida
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))

#"Compilando" = instanciando la RNN con su función de pérdida y optimización
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [15]:
# Generate some sample before training to know how bad it is!
generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)

P))ªª//((//g35555f5
AAA''''mmmNNNNccc!!!uuuufcc!!uuufcc!!uuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuu

"P))ªª//((//g35555f5\nAAA''''mmmNNNNccc!!!uuuufcc!!uuufcc!!uuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuumfccc!!uuuum"

**Se cargan los pesos (y el diccionario de los one-hot-vectors) en caso haya habido un entrenamiento previo**
<br>WEIGHTS debe tener el valor del nombre del archivo de "checkpoint" guardado. Por ejemplo:
<br>```WEIGHTS = "checkpoint_layer_2_hidden_250_epoch_60.hdf5"```
# LA SIGUIENTE LÍNEA SE EJECUTARÁ CADA VEZ QUE QUIERA ENTRENAR MÁS AL MODELO

In [17]:
#Se cargan los pesos de un entrenamiento previo (si se desea restaurar una ejecucion)
#Se calcula el numero de epocas en base al nombre del archivo
#Se carga el diccionario de caracteres (one-hot-vectors) para la generacion

### WEIGHTS = "checkpoint_layer_2_hidden_250_epoch_60.hdf5" 

if not WEIGHTS == '':
    model.load_weights(WEIGHTS)
    nb_epoch = int(WEIGHTS[WEIGHTS.rfind('_') + 1:WEIGHTS.find('.')])
    with open('ix_to_char.pickle', 'rb') as handle:
        ix_to_char = pickle.load(handle)
else:
    #Si se va a empezar de 0:
    nb_epoch = 0

### Entrenamiento

In [18]:
# Training if there is no trained weights specified

#Esta es la iteración importante
#Pueden cambiar la condición para que termine en un determinado numero de epochs.
while True:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    #Ajuste del modelo, y entrenamiento de 1 epoca
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=1)
    nb_epoch += 1
    #Generacion de un texto al final de la epoca
    generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
    #Pueden modificar esto para tener más checkpoints
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))
    if nb_epoch == 50
        break



Epoch: 0

Epoch 1/1
63924/63924 [==============================] - 805s 13ms/step - loss: 2.1058
ver the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of the count of th

Epoch: 1

Epoch 1/1
63924/63924 [==============================] - 766s 12ms/step - loss: 1.5385
  the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to the same to 

KeyboardInterrupt: 

### Generación de texto
Si instancian el modelo y sus parametros (ejecutando algunas celdas preliminares), y tienen los 2 archivos requeridos (.pickle y .hdf5) pueden generar el texto.

Si es que se va a cargar un checkpoint, ejecutar las siguientes dos líneas.

In [19]:
# Creating training data
X, y, VOCAB_SIZE, ix_to_char = load_data(DATA_DIR, SEQ_LENGTH)

Data length: 3196232 characters
Vocabulary size: 86 characters
['.', '2', 'v', 'a', 'B', '!', '(', 'W', 'u', 'q', 'o', 'U', 'b', 'D', 'z', 'f', 'n', '¤', ' ', 'r', '?', 'C', 't', '9', '/', 'k', 'Q', 'S', '\xa0', 'x', 'Z', 'L', '»', '©', 'N', 'K', '*', 'ï', "'", 'y', 'I', '¿', 'l', ';', ':', 's', 'V', 'A', 'g', 'G', '0', 'i', 'Ã', '4', 'Y', 'M', '=', 'H', '1', 'w', 'j', '6', '"', 'P', '3', '-', '5', 'd', 'm', 'J', ',', '8', 'h', 'T', 'F', '\n', 'c', 'p', ')', 'X', 'ª', 'O', 'R', 'e', '7', 'E']
Number of sequences: 63924


In [ ]:
# Creating and compiling the Network
model = Sequential()

#Añadiendo las capas LSTM
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
#Añadiendo la operacion de salida
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))

#"Compilando" = instanciando la RNN con su función de pérdida y optimización
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [19]:
#Cuidar de no reemplazar el pickle original
with open('ix_to_char.pickle', 'rb') as handle:
    ix_to_char = pickle.load(handle)
    
WEIGHTS = "checkpoint_layer_2_hidden_250_epoch_50.hdf5"
# Loading the trained weights
model.load_weights(WEIGHTS)
generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
print('\n\n')

u(ROFoPZFSZ'PR

KeyError: 85